Programming Assignment: Рекомендательные системы
Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

Задание

    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
    Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

    Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
    Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.



In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("coursera_sessions_train.txt", sep=';', header=None)

In [4]:
data.head()

,0,1
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [12]:
data.columns = ['view', 'buy']

In [13]:
view_freq = {}

In [14]:
for row in data.view.values:
    r = list(row.split(','))
    for i in r:
        if view_freq.get(i) == None:
            view_freq[i] = 1
        else:
            view_freq[i] += 1

In [15]:
keys = sorted(view_freq, key=view_freq.get, reverse=True)
for k in keys[:10]:
    print k, ' ', view_freq[k]

73   677
158   641
204   396
262   387
162   318
7   312
137   306
1185   284
6   283
170   280


In [17]:
buy_freq = {}
for row in data.buy.dropna().values:
    r = list(row.split(','))
    for i in r:
        if buy_freq.get(i) == None:
            buy_freq[i] = 1
        else:
            buy_freq[i] += 1

In [19]:
keys1 = sorted(buy_freq, key=buy_freq.get, reverse=True)
for k in keys1[:10]:
    print k, ' ', buy_freq[k]

158   14
204   12
3324   11
73   11
977   10
5569   10
3149   10
1181   9
162   8
1852   7


In [20]:
def drop_dup(s):
    l = list(s.split(','))
    res = []
    for i in l:
        if i not in res:
            res.append(i)
            
    return ",".join(res)

In [21]:
filtered = data.dropna()
filtered.view = filtered.view.apply(drop_dup)
filtered.head()

C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,view,buy
7,"59,60,61,62,63,64,65,66,67,68","67,60,63"
10,"84,85,86,87,88,89,90,91,92,93",86
19,"138,198,199,127",199
30,"303,304,305,306,307,308,309,310,311,312",303
33,"352,353",352


In [22]:
def getSorted(s, d):
    l = [x.strip() for x in s.split(',')]
    tuples = []
    for i in l:
        if d.get(i) == None:
            tuples.append((0, i))
        else:
            tuples.append((d[i], i))
    ht = lambda x, y : y[0] - x[0] if x[0] != y[0] else l.index(x[1]) - l.index(y[1])
    return sorted(tuples, cmp=ht)

def suggest5view(s):
    rec = getSorted(s, view_freq)
    if len(rec) > 5:
        return map(lambda x : x[1], rec[:5])
    else:
        return map(lambda x : x[1], rec)

def suggest5buy(s):
    rec = getSorted(s, buy_freq)
    if len(rec) > 5:
        return map(lambda x : x[1], rec[:5])
    else:
        return map(lambda x : x[1], rec)

In [24]:
ap1 = []
ap5 = []
ar1 = []
ar5 = []
ap1buy = []
ap5buy = []
ar1buy = []
ar5buy = []
for row in filtered.iterrows():
    res = suggest5view(row[1]['view'])
    one = res[0]
    buy_l = [x.strip() for x in row[1]['buy'].split(',')]
    p1 = 1 if one in buy_l else 0
    r1 = (float(p1)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res])
    p5 = float(nom) / 5
    r5 = float(nom) / len(buy_l)
    ap1.append(p1)
    ap5.append(p5)
    ar1.append(r1)
    ar5.append(r5)
    
    res_buy = suggest5buy(row[1]['view'])
    one_buy = res_buy[0]
    p1_buy = 1 if one_buy in buy_l else 0
    r1_buy = (float(p1_buy)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res_buy])
    p5_buy = float(nom) / 5
    r5_buy = float(nom) / len(buy_l)
    ap1buy.append(p1_buy)
    ap5buy.append(p5_buy)
    ar1buy.append(r1_buy)
    ar5buy.append(r5_buy)

In [25]:
def writeArr(arr, filename):
    with open(filename, "w") as f:
        res = " ".join(map(str, arr))
        f.write(res)

In [35]:
arg = [sum(ar1) / float(len(ar1)),
       sum(ap1) / float(len(ap1)),
       sum(ar5) / float(len(ar5)),
       sum(ap5) / float(len(ap5))]
writeArr(arg, "train_res_view.txt")

In [36]:
arg = [sum(ar1buy) / float(len(ar1buy)),
       sum(ap1buy) / float(len(ap1buy)),
       sum(ar5buy) / float(len(ar5buy)),
       sum(ap5buy) / float(len(ap5buy))]
writeArr(arg, "train_res_buy.txt")

In [37]:
test = pd.read_csv("coursera_sessions_test.txt", sep=';', header=None)
test[:5]

,0,1
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [38]:
test.columns = ['view', 'buy']
test_filt = test.dropna()
test_filt.view = test_filt.view.apply(drop_dup)
test_filt.head()

,view,buy
7,"63,68,69,70,66,61,59","66,63"
14,"158,159,160,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372","371,373"
40,422,422


In [39]:
ap1 = []
ap5 = []
ar1 = []
ar5 = []
ap1buy = []
ap5buy = []
ar1buy = []
ar5buy = []
for row in test_filt.iterrows():
    res = suggest5view(row[1]['view'])
    one = res[0]
    buy_l = [x.strip() for x in row[1]['buy'].split(',')]
    p1 = 1 if one in buy_l else 0
    r1 = (float(p1)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res])
    p5 = float(nom) / 5
    r5 = float(nom) / len(buy_l)
    ap1.append(p1)
    ap5.append(p5)
    ar1.append(r1)
    ar5.append(r5)
    
    res_buy = suggest5buy(row[1]['view'])
    one_buy = res_buy[0]
    p1_buy = 1 if one_buy in buy_l else 0
    r1_buy = (float(p1_buy)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res_buy])
    p5_buy = float(nom) / 5
    r5_buy = float(nom) / len(buy_l)
    ap1buy.append(p1_buy)
    ap5buy.append(p5_buy)
    ar1buy.append(r1_buy)
    ar5buy.append(r5_buy)  


In [41]:
arg = [sum(ar1) / float(len(ar1)),
       sum(ap1) / float(len(ap1)),
       sum(ar5) / float(len(ar5)),
       sum(ap5) / float(len(ap5))]
writeArr(arg, "test_res_view.txt")

In [42]:
arg = [sum(ar1buy) / float(len(ar1buy)),
       sum(ap1buy) / float(len(ap1buy)),
       sum(ar5buy) / float(len(ar5buy)),
       sum(ap5buy) / float(len(ap5buy))]
writeArr(arg, "test_res_buy.txt")

In [31]:
check = {}
for row in data.buy.dropna()[:50].values:
    l = [x.strip() for x in str(row).split(',')]
    print l
    for i in l:
        if check.get(i) == None:
            check[i] = 1
        else:
            check[i] += 1
            
keys = sorted(check, key=check.get, reverse=True)
for k in keys[:10]:
    print k, ' ', check[k]

['67', '60', '63']
['86']
['199']
['303']
['352']
['519']
['603', '604', '602', '599', '605', '606', '600']
['690', '688']
['851']
['879']
['1118']
['1545']
['1727']
['99']
['1907']
['1959']
['1998']
['2013']
['2019']
['2462']
['2520']
['2543']
['1526']
['2764']
['2857', '2853', '2852']
['2920']
['2930']
['3033', '3026', '3032', '3031']
['3102', '3100', '1260']
['3204', '3207', '3217']
['3267']
['3443']
['3529']
['3584', '3564', '3612', '3623', '3645', '3561']
['1662']
['3741']
['3979']
['4216']
['4246']
['4298']
['3066']
['4512']
['4750']
['4815']
['5058']
['5066']
['5134', '5135']
['5183', '970']
['5209']
['805']
605   1
604   1
3612   1
606   1
199   1
600   1
603   1
602   1
599   1
2920   1
